In [ ]:
import retinanalysis as ra
import retinanalysis.schema as schema
import numpy as np
from matplotlib.patches import Ellipse
import matplotlib.pyplot as plt
import pandas as pd

ra.djconnect()


## Step 1: Query protocol across all experiments

In [ ]:
protocols_to_query = ['movingletters', 'presentmatfiles']
experiment_query = ra.get_datasets_from_protocol_names(protocols_to_query)
display(experiment_query)

In [ ]:
noise_vcd = ra.get_analysis_vcd('20250514C', 'chunk1', 'kilosort2.5')
test_vcd = ra.get_protocol_vcd('20250514C', 'data003', 'kilosort2.5')

noise_ids = noise_vcd.get_cell_ids()
test_ids = test_vcd.get_cell_ids()

noise_eis = [noise_vcd.get_ei_for_cell(id).ei for id in noise_ids]
test_eis = [test_vcd.get_ei_for_cell(id).ei for id in test_ids]

In [ ]:
corr_vals = ra.ei_corr(noise_vcd, noise_vcd, method = 'full')

fig, ax = plt.subplots(figsize=(10,10))
ax.imshow(corr_vals);

In [ ]:
n_removed_channels = 1

max_noise_channels = [np.argmax(np.max(ei, axis = 1)) for ei in noise_eis]
max_test_channels = [np.argmax(np.max(ei, axis = 1)) for ei in test_eis]

print(max_noise_channels)
print(max_test_channels)

## Select experiment and data file of interest

In [ ]:
exp_name = experiment_query.loc[0, 'exp_name']
datafile_name = experiment_query.loc[0,'datafile_name']

# Create stim, response, analysis_chunk and pipeline class
stim = ra.StimBlock(exp_name, datafile_name)
response = ra.ResponseBlock(exp_name, datafile_name)
analysis_chunk = ra.AnalysisChunk(exp_name, stim.nearest_noise_chunk)
pipeline = ra.MEAPipeline(stim, response, analysis_chunk)

print(analysis_chunk.chunk_name)

In [ ]:
full_corr = pipeline.classification_transfer('chunk3', output_typing_file = 'fullCorr_classification.txt', use_isi = True, use_timecourse = True, method = 'full')
space_corr = pipeline.classification_transfer('chunk3', output_typing_file = 'spaceCorr_classification.txt', use_isi = True, use_timecourse = True, method = 'space')
power_corr = pipeline.classification_transfer('chunk3', output_typing_file = 'powerCorr_classification.txt', use_isi = True, use_timecourse = True, method = 'power')
all_corr = pipeline.classification_transfer('chunk3', output_typing_file = 'allCorr_classification.txt', use_isi = True, use_timecourse = True, method = 'all')

In [ ]:


pd.set_option('display.max_rows', None)

def add_matches(input_df, column_name, match_dict, protocol_ids):

    for id in protocol_ids:
        if id in match_dict:
            pass
        else:
            match_dict[id] = 0

    input_df[column_name] = match_dict

    return input_df

corr_compare = analysis_chunk.df_cell_params[['cell_id', 'typing_file_0']].copy()
protocol_ids = corr_compare['cell_id'].values
corr_compare = corr_compare.set_index('cell_id')

corr_compare  = add_matches(corr_compare, 'full_corr', full_corr, protocol_ids)
corr_compare = add_matches(corr_compare, 'space_corr', space_corr, protocol_ids)
corr_compare = add_matches(corr_compare, 'power_corr', power_corr, protocol_ids)
corr_compare = add_matches(corr_compare, 'all_corr', all_corr, protocol_ids)

display(corr_compare.query('typing_file_0 == "OffP"'))

In [ ]:
display(analysis_chunk.typing_files)

In [ ]:
cell_ids = response.df_spike_times.iloc[0:50].index.values
# cell_types = ['OnP', 'OffP']
# cell_ids = list(response.df_spike_times.query('cell_type == @cell_types').index.values)
# display(response.df_spike_times.query('cell_type == @cell_types'))

axes = pipeline.plot_rfs(cell_ids = cell_ids, units = 'microns');


In [ ]:
epoch_params = ['imageOrder']
image_order = [lst for lst in stim.df_epochs['imageOrder']]
image_order = np.array(image_order)

# display(stim.df_epochs.columns)

images_per_epoch = stim.df_epochs.loc[0,'epoch_parameters']['imagesPerEpoch']
num_epochs = len(stim.df_epochs)

# epochs_to_see = [1, 2, 5, 245, 6, 232]
# display(stim.df_epochs[['imageOrder', 'matFile']].query('epoch_index == @epochs_to_see'))

cell_of_interest = 6
cell_1_times = response.df_spike_times.query('cell_id == 1')['spike_times'].values

OffP_noise_ids = analysis_chunk.cell_params_df.query('typing_file_0 == "OffP"')['cell_id'].values
OffP_protocol_ids = [pipeline.match_dict[cell] for cell in OffP_noise_ids if cell in pipeline.match_dict.keys()]
OffP_responses = response.df_spike_times.query('cell_id == @OffP_protocol_ids')
OffP_spike_times = OffP_responses['spike_times'].values

test = np.empty((OffP_spike_times.shape[0], OffP_spike_times[0].shape[0]), dtype = object)
for i in range(test.shape[0]):
    test[i,:] = OffP_spike_times[i]

print(test.shape)
fig, ax = plt.subplots()

epochs_to_plot = np.arange(0,9)
ax.eventplot(test[:,0], color = 'k', linewidths = 0.8, linelengths = 0.8)
ax.set_xlabel("Time (ms)")
ax.set_ylabel("Epoch")


In [ ]:
cell_types = ['OnP', 'OffP']
cell_ids = [1, 3, 9, 10, 38, 42, 121,]

cts = list(response.df_spike_times['cell_type'].unique())

display(cts)
test = [key for key, val in pipeline.match_dict.items()]
print(test)


protocol_ids = response.df_spike_times.query('cell_type == @cell_types').index.values
ids_to_plot = [key for key, val in pipeline.match_dict.items() if (val in protocol_ids
                                                           and val in cell_ids)]

corr_ids = [val for key, val in pipeline.match_dict.items() if (val in protocol_ids
                                                        and val in cell_ids)]
print(ids_to_plot)
print(corr_ids)

In [ ]:

def get_ells(analysis_chunk, cell_types, std_scaler = 1.6, units = None):
    
    if units is not None:
        if 'microns' in units.lower():
            scale_factor = analysis_chunk.microns_per_stixel
        elif 'pixels' in units.lower():
            scale_factor = analysis_chunk.pixels_per_stixel
        else:
            scale_factor = 1
    else:
        scale_factor = 1

    ells = []
    for idx, ct in enumerate(cell_types):
        ell_dict = dict()
        cell_df = analysis_chunk.cell_params_df.query('typing_file_0 == @ct')
        rf_params = analysis_chunk.rf_params
        cell_ids = cell_df['cell_id'].values
        for id in cell_ids:
            ell_dict[id] = Ellipse(xy=(rf_params[id]['center_x']*scale_factor,
                                    rf_params[id]['center_y']*scale_factor),
                                    width = rf_params[id]['std_x']*std_scaler*scale_factor,
                                    height = rf_params[id]['std_y']*std_scaler*scale_factor,
                                    angle = rf_params[id]['rot'],
                                    facecolor= f'C{idx}', edgecolor= f'C{idx}',
                                    alpha = 0.7)
        ells.append(ell_dict)
    
    return ells, scale_factor

units = 'pixels'
cell_types = ['OffP', 'OnP', 'OffM', 'OnM', 'SBC']
all_ells, scale_factor = get_ells(analysis_chunk, cell_types, units = units)

fig, ax = plt.subplots(nrows=1, ncols = len(cell_types), figsize = (18, 12/len(cell_types)))

for idx, ct in enumerate(cell_types):
    for id in all_ells[idx]:
        ax[idx].add_patch(all_ells[idx][id])

    ax[idx].set_xlim(0,analysis_chunk.numXChecks * scale_factor)
    ax[idx].set_ylim(0,analysis_chunk.numYChecks * scale_factor)
    if units is not None:
        ax[idx].set_ylabel(units.lower())
        ax[idx].set_xlabel(units.lower())

fig.tight_layout()

